In [1]:
import math
import logging
FORMAT = '[%(name)s:%(levelname)s]  %(message)s'
logging.basicConfig(level=logging.DEBUG, format=FORMAT)
logger = logging.getLogger('dbg')

def dprint(s):
    logger.debug(s)

def iprint(s):
    logger.info(s)

logger.setLevel(logging.INFO)

## B-trees

B-Trees are a self-balancing search-trees with fast search, insertion and deletion, where nodes can have many children.

**Worst case Search/Insert/Delete** $\Rightarrow O( \log n )$, storage is $\Theta (n)$

<img src="media/btree.png" alt="drawing" width="450"/>

The key idea is to optimize for realistic memory hierarchy by making the number of children as large as possible, but so that each node remains within a single disc block and requires just one memory access. A shallow-wide tree requires few accesses to main memory. For a tree with 200 children (199 keys per node) depth three contains $1 + 200 + 200^2 + 200^3 = 8,040,201$ nodes $= 1,600,000,000$ Keys!.

### Definition

When a B-tree has $t=2$ and allows key counts of 1 to 3 supporting nodes of 2 to 4, it is called a **2-4** or **2-3-4** Tree.

A B-tree with minimum degree `t`:
- Nodes have attributes `keys` (list) and `is_leaf`.
- Internal nodes have `len(keys) + 1` children where:
- The keys of the node **separate its children**.
- $ t - 1\leq $ node `keys` $ \leq 2t -1$ (except for the root). 
- $ t \leq $ `children` $ \leq 2t$ (except for the root). 

- ALl leaves have the same depth.
